#### SARIMA gridsearch

The error looks better than 24 and we can try out different K's but usually if the data isnt too much - we can create sort of a hyperparameter tuning function to find us best parameters

In [ ]:
from itertools import product

def grid_search_sarima_exog(
    df, splits,
    K_list=(2,3,4),                      
    orders=((1,0,0), (2,0,0), (1,0,1), (2,0,1)),
    seasonal_orders=((1,0,1,24),(1,0,0,24)), #,(0,0,1,24)
    min_train_size=24*30, max_train_size=24*90,
    weekly_period=168,
    metric='mape'                            
):
    exog_cache = {K: fourier_df(df.index, weekly_period, K) for K in K_list}

    rows = []
    for K in K_list:
        exog = exog_cache[K]
        for order, seas in product(orders, seasonal_orders):
            sarima_exog_model_fit, sarima_exog_all_metrics, sarima_exog_avg_metrics = train_sarima_exog_on_splits(
                splits, order=order, seasonal_order=seas,
                min_train_size=min_train_size, max_train_size=max_train_size, exog=exog, iter=f"{K}_{order}_{seas}"
            )
            rows.append({
                "K": K, "order": order, "seasonal_order": seas, **sarima_exog_avg_metrics
            })
            print(f"K={K} order={order} seas={seas} | MAPE={sarima_exog_avg_metrics.get('mape',np.nan):.2f} RMSE={sarima_exog_avg_metrics.get('rmse',np.nan):.0f}")

    if not rows:
        raise RuntimeError("Grid search produced no results")

    leaderboard = pd.DataFrame(rows)

    if metric.lower() in ("mape","rmse","mae"):
        leaderboard = leaderboard.sort_values(metric).reset_index(drop=True)
    elif metric.lower() == "r2":
        leaderboard = leaderboard.sort_values(metric, ascending=False).reset_index(drop=True)
    else:
        raise ValueError("metric must be one of: 'mape','rmse','mae','r2'")

    best_row = leaderboard.iloc[0].to_dict()
    return best_row, leaderboard

In [ ]:
custom_expanding_window_splits_selected = [custom_expanding_window_splits[-1]]

In [ ]:
best, board = grid_search_sarima_exog(
    df=df,
    splits=custom_expanding_window_splits_selected,
    K_list=(3,4),
    orders=((1,0,0),(1,0,1),(2,0,1)),
    seasonal_orders=((1,0,1,24),(1,0,0,24)),
    min_train_size=24*30,
    max_train_size=24*100,          
    weekly_period=168,
    metric='mape'               
)
print("Best config:", best)
board.head()

this could take quite long - as it evaluates all the combinations, an alternative is auto arima

##### Load saved & Eval

In [ ]:
import glob
import re
from statsmodels.tsa.statespace.sarimax import SARIMAXResults

custom_expanding_window_splits_selected = [custom_expanding_window_splits[-1]]

for i, (train, test) in enumerate(custom_expanding_window_splits_selected):
    model_files = [f for f in glob.glob('./data/output/models/SARIMA_exog/SARIMA_exog_fold_1_*.pkl')
               if not re.search(r"\(0, 0, 1, 24\)\.pkl$", f) and not re.search(r"_2_.*_\(1, 0, 0, 24\)\.pkl$", f)]
    agg = []
    
    for file in model_files:
        try:
            k = int(file.split('_')[-3]) 
            print(f"\nProcessing model file: {file} with K={k}")

            exog_full = fourier_df(df.index, period=168, K=k)
            exog_test = exog_full.loc[test.index]

            model_fit = SARIMAXResults.load(file)

            forecast_obj = model_fit.get_forecast(steps=len(test), exog=exog_test)
            forecast_mean = forecast_obj.predicted_mean
            pred_ci = forecast_obj.conf_int()

            y_true = test['demand'].values
            y_pred = forecast_mean.values

            metrics = evaluate_forecast( y_true=y_true, y_pred=y_pred, model_name=file.split('/')[-1], plot=True, train=train['demand'], test=test['demand'], forecast_mean=forecast_mean, pred_ci=pred_ci, fold=i + 1)

            agg.append({
                "file": file,
                "K": k,
                "order": model_fit.model.order,
                "seasonal_order": model_fit.model.seasonal_order,
                "rmse": metrics["rmse"],
                "mae": metrics["mae"],
                "mape": metrics["mape"],
                "r2": metrics["r2"],
                "aic": model_fit.aic,
                "bic": model_fit.bic
            })
            
        except Exception as e:
            print(f"Failed to process {file} on fold {i + 1}: {e}")


In [ ]:
agg_df = pd.DataFrame(agg)
grouped = agg_df.groupby(["file", "K", "order", "seasonal_order", "aic", "bic"])

rows = []
for (file, K, order, seasonal_order, aic, bic), group in grouped:
    if group["rmse"].isna().all():
        continue

    rows.append({
        "file": file,
        "K": K,
        "order": order,
        "seasonal_order": seasonal_order,
        "aic": aic,
        "bic": bic,
        "folds": len(group),
        "mean_rmse": group["rmse"].mean(),
        "std_rmse": group["rmse"].std(),
        "mean_mae": group["mae"].mean(),
        "mean_mape": group["mape"].mean(),
        "mean_r2": group["r2"].mean(),
    })

summary = pd.DataFrame(rows)
summary = summary.sort_values(by=["mean_rmse", "mean_mape", "aic"],
                              ascending=[True, True, True]).reset_index(drop=True)
best = summary.iloc[0]
print("\n===== Best Model (by mean RMSE across folds) =====")
print(f"File:           {best['file']}")
print(f"K:              {best['K']}")
print(f"Order:          {best['order']}")
print(f"Seasonal order: {best['seasonal_order']}")
print(f"AIC / BIC:      {best['aic']:.3f} / {best['bic']:.3f}")
print(f"Folds:          {int(best['folds'])}")
print(f"Mean RMSE:      {best['mean_rmse']:.4f} (± {best['std_rmse']:.4f})")
print(f"Mean MAE:       {best['mean_mae']:.4f}")
print(f"Mean MAPE:      {best['mean_mape']:.2f}%")
print(f"Mean R²:        {best['mean_r2']:.4f}")

print("\n===== All Models Summary (sorted) =====")
print(summary.to_string(index=False))


===== Best Model (by mean RMSE across folds) =====
File:           ./data/output/models/SARIMA_exog\SARIMA_exog_fold_1_4_(2, 0, 1)_(1, 0, 1, 24).pkl
K:              4
Order:          (2, 0, 1)
Seasonal order: (1, 0, 1, 24)
AIC / BIC:      35171.386 / 35252.199
Folds:          1
Mean RMSE:      4995.7568 (± nan)
Mean MAE:       3853.0845
Mean MAPE:      11.43%
Mean R²:        0.5361

===== All Models Summary (sorted) =====
                                                                             file  K     order seasonal_order          aic          bic  folds    mean_rmse  std_rmse     mean_mae  mean_mape    mean_r2
./data/output/models/SARIMA_exog\SARIMA_exog_fold_1_4_(2, 0, 1)_(1, 0, 1, 24).pkl  4 (2, 0, 1)  (1, 0, 1, 24) 35171.386257 35252.198899      1  4995.756817       NaN  3853.084525  11.428725   0.536104
./data/output/models/SARIMA_exog\SARIMA_exog_fold_1_3_(2, 0, 1)_(1, 0, 1, 24).pkl  3 (2, 0, 1)  (1, 0, 1, 24) 35168.485158 35237.753137      1  5003.337317       NaN  3856

#### HW Grid search

In [ ]:
from itertools import product
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_error, r2_score

def to_series(x):
    if isinstance(x, pd.DataFrame):
        return x["demand"] if "demand" in x.columns else x.squeeze()
    return pd.Series(x)

def rmse(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    return float(np.sqrt(((y_true - y_pred) ** 2).mean()))

def safe_mape(y_true, y_pred, eps=1e-8):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    denom = np.where(np.abs(y_true) < eps, eps, y_true)
    return float(np.mean(np.abs((y_true - y_pred) / denom)) * 100)

def evaluate_hw(train, test, trend, seasonal, seasonal_periods, damped):

    train_s = to_series(train).astype(float)
    test_s  = to_series(test).astype(float)

    model = ExponentialSmoothing(
        train_s,
        trend=trend,
        damped_trend=damped,
        seasonal=seasonal,
        seasonal_periods=seasonal_periods,
    )
    fit = model.fit(optimized=True)
    pred = fit.forecast(len(test_s))

    return {
        "trend": trend,
        "seasonal": seasonal,
        "seasonal_periods": seasonal_periods,
        "damped": damped,
        "rmse": rmse(test_s, pred),
        "mae": mean_absolute_error(test_s, pred),
        "mape": safe_mape(test_s, pred),
        "r2": r2_score(test_s, pred),
    }

trend_options = [None, "add", "mul"]
seasonal_options = ["add", "mul"]       
seasonal_periods_options = [24, 168]    
damped_options = [False, True]

results = []
(train_split, test_split) = custom_expanding_window_splits_selected[0]

for trend, seasonal, sp, damped in product(trend_options, seasonal_options, seasonal_periods_options, damped_options):
    try:
        metrics = evaluate_hw(train_split, test_split, trend, seasonal, sp, damped)
        results.append(metrics)
    except Exception as e:
        if trend is None and damped:
            print("Invalid combo: damped=True requires trend to be 'add' or 'mul'")
        print(f"Failed for trend={trend}, seasonal={seasonal}, sp={sp}, damped={damped}: {e}")

df_results = pd.DataFrame(results).sort_values(["rmse", "mape"]).reset_index(drop=True)

print("\nTop candidates:")
print(df_results.head(10).to_string(index=False))

best = df_results.iloc[0]
print("\n===== Best Holt-Winters Params =====")
print(f"trend={best.trend}, seasonal={best.seasonal}, sp={int(best.seasonal_periods)}, damped={bool(best.damped)}")
print(f"RMSE={best.rmse:.4f}, MAE={best.mae:.4f}, MAPE={best.mape:.2f}%, R²={best.r2:.4f}")


Invalid combo: damped=True requires trend to be 'add' or 'mul'.
Failed for trend=None, seasonal=add, sp=24, damped=True: Can only dampen the trend component
Invalid combo: damped=True requires trend to be 'add' or 'mul'.
Failed for trend=None, seasonal=add, sp=168, damped=True: Can only dampen the trend component
Invalid combo: damped=True requires trend to be 'add' or 'mul'.
Failed for trend=None, seasonal=mul, sp=24, damped=True: Can only dampen the trend component
Invalid combo: damped=True requires trend to be 'add' or 'mul'.
Failed for trend=None, seasonal=mul, sp=168, damped=True: Can only dampen the trend component

Top candidates:
trend seasonal  seasonal_periods  damped        rmse         mae      mape       r2
  mul      add               168    True 5481.196834 4196.436417 12.321977 0.441570
  add      mul               168    True 5628.139778 4192.594634 11.885820 0.411227
  mul      mul               168    True 5629.371384 4193.426584 11.887151 0.410970
 None      mul   

#### Prophet Grid Search

In [ ]:
from itertools import product
from prophet import Prophet
    
def to_prophet(df):
    if isinstance(df, pd.DataFrame):
        y = df["demand"] if "demand" in df.columns else df.squeeze()
        return pd.DataFrame({"ds": df.index, "y": y.astype(float).values})

    s = pd.Series(df)
    return pd.DataFrame({"ds": s.index, "y": s.astype(float).values})

def rmse(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    return float(np.sqrt(np.mean((y_true - y_pred) ** 2)))

def safe_mape(y_true, y_pred, eps=1e-8):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    denom = np.where(np.abs(y_true) < eps, eps, y_true)
    return float(np.mean(np.abs((y_true - y_pred) / denom)) * 100)

def build_prophet(params):
    m = Prophet(
        growth=params.get("growth", "linear"),
        seasonality_mode=params.get("seasonality_mode", "additive"),
        changepoint_prior_scale=params.get("cps", 0.1),
        changepoint_range=params.get("changepoint_range", 0.9),
        n_changepoints=params.get("n_changepoints", 25),
        seasonality_prior_scale=params.get("sps", 10.0),
        holidays_prior_scale=params.get("hps", 10.0),
        interval_width=0.95,
    )

    m.add_seasonality(name="daily",  period=1,        fourier_order=params.get("daily_fourier", 10))      # 1 day
    m.add_seasonality(name="weekly", period=7,        fourier_order=params.get("weekly_fourier", 6))      # 7 days
    m.add_seasonality(name="yearly", period=365.25,   fourier_order=params.get("yearly_fourier", 10))     # 1 year

    if params.get("country_holidays"):
        m.add_country_holidays(country_name=params["country_holidays"])

    for reg in params.get("regressors", []):
        m.add_regressor(reg)

    return m

def fit_predict_one_split(train_df, test_df, params):

    m = build_prophet(params)
    train_p = to_prophet(train_df).dropna()
    test_p  = to_prophet(test_df).dropna()

    m.fit(train_p)

    future = pd.DataFrame({"ds": test_p["ds"]})
    fcst = m.predict(future)
    y_true = test_p["y"].values
    y_pred = fcst["yhat"].values

    return {
        "rmse": rmse(y_true, y_pred),
        "mae": float(np.mean(np.abs(y_true - y_pred))),
        "mape": safe_mape(y_true, y_pred),
        "r2": 1.0 - (np.sum((y_true - y_pred)**2) / np.sum((y_true - np.mean(y_true))**2) if np.var(y_true) > 0 else np.nan),
    }


In [ ]:
def prophet_grid_search(
    splits,
    cps=(0.01, 0.1, 0.3),                 
    sps=(5.0, 10.0),                      
    seasonality_mode=("additive", "multiplicative"),
    daily_fourier=(10,),                 
    weekly_fourier=(6,),
    yearly_fourier=(10,),
    n_changepoints=(25,),
    changepoint_range=(0.9,),
    country_holidays=("US", None),        
    regressors=None,                      
):
    results = []
    grid = product(cps, sps, seasonality_mode, daily_fourier, weekly_fourier, yearly_fourier,
                   n_changepoints, changepoint_range, country_holidays)

    for (cps_, sps_, mode_, d_f, w_f, y_f, ncp, cpr, hol) in grid:
        params = dict(
            cps=cps_, sps=sps_, seasonality_mode=mode_,
            daily_fourier=d_f, weekly_fourier=w_f, yearly_fourier=y_f,
            n_changepoints=ncp, changepoint_range=cpr,
            country_holidays=hol, regressors=(regressors or [])
        )

        fold_metrics = []
        try:
            for train_df, test_df in splits:
                m = fit_predict_one_split(train_df, test_df, params)
                fold_metrics.append(m)

            avg = {k: float(np.mean([fm[k] for fm in fold_metrics if np.isfinite(fm[k])])) for k in ["rmse","mae","mape","r2"]}
            results.append({**params, **{f"mean_{k}": v for k,v in avg.items()}})
            print(f"{params}  ->  RMSE={avg['rmse']:.2f}, MAPE={avg['mape']:.2f}%")

        except Exception as e:
            print(f"× Skipping {params}: {e}")


    summary = pd.DataFrame(results).sort_values(["mean_rmse","mean_mape"]).reset_index(drop=True)
    best = summary.iloc[0]

    print("\n===== Best Prophet (by mean RMSE across folds) =====")
    print(f"mode={best['seasonality_mode']}, cps={best['cps']}, sps={best['sps']}, n_changepoints={int(best['n_changepoints'])}, "
          f"cpr={best['changepoint_range']}, dFO={int(best['daily_fourier'])}, wFO={int(best['weekly_fourier'])}, "
          f"yFO={int(best['yearly_fourier'])}, holidays={best['country_holidays']}")
    print(f"RMSE={best['mean_rmse']:.2f}, MAE={best['mean_mae']:.2f}, MAPE={best['mean_mape']:.2f}%, R²={best['mean_r2']:.3f}")

    return best, summary

In [ ]:
prophet_grid_search(splits=custom_expanding_window_splits_selected)

20:24:41 - cmdstanpy - INFO - Chain [1] start processing
20:25:28 - cmdstanpy - INFO - Chain [1] done processing


✓ {'cps': 0.01, 'sps': 5.0, 'seasonality_mode': 'additive', 'daily_fourier': 10, 'weekly_fourier': 6, 'yearly_fourier': 10, 'n_changepoints': 25, 'changepoint_range': 0.9, 'country_holidays': 'US', 'regressors': []}  ->  RMSE=4126.69, MAPE=8.93%


20:25:43 - cmdstanpy - INFO - Chain [1] start processing
20:26:04 - cmdstanpy - INFO - Chain [1] done processing


✓ {'cps': 0.01, 'sps': 5.0, 'seasonality_mode': 'additive', 'daily_fourier': 10, 'weekly_fourier': 6, 'yearly_fourier': 10, 'n_changepoints': 25, 'changepoint_range': 0.9, 'country_holidays': None, 'regressors': []}  ->  RMSE=4112.03, MAPE=8.91%


20:26:17 - cmdstanpy - INFO - Chain [1] start processing
20:26:59 - cmdstanpy - INFO - Chain [1] done processing


...

20:58:43 - cmdstanpy - INFO - Chain [1] start processing
21:00:37 - cmdstanpy - INFO - Chain [1] done processing


✓ {'cps': 0.3, 'sps': 10.0, 'seasonality_mode': 'multiplicative', 'daily_fourier': 10, 'weekly_fourier': 6, 'yearly_fourier': 10, 'n_changepoints': 25, 'changepoint_range': 0.9, 'country_holidays': 'US', 'regressors': []}  ->  RMSE=10099.67, MAPE=28.70%


21:00:46 - cmdstanpy - INFO - Chain [1] start processing
21:02:10 - cmdstanpy - INFO - Chain [1] done processing


✓ {'cps': 0.3, 'sps': 10.0, 'seasonality_mode': 'multiplicative', 'daily_fourier': 10, 'weekly_fourier': 6, 'yearly_fourier': 10, 'n_changepoints': 25, 'changepoint_range': 0.9, 'country_holidays': None, 'regressors': []}  ->  RMSE=10083.48, MAPE=28.61%

===== Best Prophet (by mean RMSE across folds) =====
mode=multiplicative, cps=0.01, sps=10.0, n_changepoints=25, cpr=0.9, dFO=10, wFO=6, yFO=10, holidays=US
RMSE=3909.99, MAE=2928.94, MAPE=8.97%, R²=0.716


(cps                            0.01
 sps                            10.0
 seasonality_mode     multiplicative
 daily_fourier                    10
 weekly_fourier                    6
 yearly_fourier                   10
 n_changepoints                   25
 changepoint_range               0.9
 country_holidays                 US
 regressors                       []
 mean_rmse               3909.992837
 mean_mae                2928.942684
 mean_mape                  8.967069
 mean_r2                    0.715836
 Name: 0, dtype: object,
      cps   sps seasonality_mode  daily_fourier  weekly_fourier  \
 0   0.01  10.0   multiplicative             10               6   
 1   0.01   5.0   multiplicative             10               6   
 2   0.01  10.0   multiplicative             10               6   
 3   0.01   5.0   multiplicative             10               6   
 4   0.10  10.0         additive             10               6   
 5   0.10   5.0         additive             10        